- La liga italiana ha tenido fama de defensiva, creo que se meten menos de dos goles de media por partido.

- ¿Se meten menos goles en Italia que en España?

- ¿Hay diferencia de goles entre ligas?, ¿en que liga se meten más goles?.

- ¿Se están metiendo menos goles en general en las 5 grandes ligas respecto el año pasado?

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

futbol_df=pd.read_csv("Futbol.csv")

ita_df=futbol_df[futbol_df["Country"]=="it ITA"]

esp_df=futbol_df[futbol_df["Country"]=="es ESP"]

fra_df=futbol_df[futbol_df["Country"]=="fr FRA"]

eng_df=futbol_df[futbol_df["Country"]=="eng ENG"]

de_df=futbol_df[futbol_df["Country"]=="de GER"]

In [11]:
ita_df.head()
esp_df.head()

#GF / MP = gole por partido
futbol_df["GPP"]=futbol_df["GF"]/futbol_df["MP"]
ita_df["GPP"]=ita_df["GF"]/ita_df["MP"]
esp_df["GPP"]=esp_df["GF"]/esp_df["MP"]
fra_df["GPP"]=fra_df["GF"]/fra_df["MP"]
eng_df["GPP"]=eng_df["GF"]/eng_df["MP"]
de_df["GPP"]=de_df["GF"]/de_df["MP"]

ita_df.head()

print("Media de goles en italia", ita_df.groupby("Country").mean()["GF"])
print("Media de goles en españa", esp_df.groupby("Country").mean()["GF"])

print("Media de goles marcados por partido en italia ",ita_df.groupby("Country").mean()["GPP"])
print("Media de goles marcados por partidos en españa",esp_df.groupby("Country").mean()["GPP"])

Media de goles en italia Country
it ITA    31.45
Name: GF, dtype: float64
Media de goles en españa Country
es ESP    29.05
Name: GF, dtype: float64
Media de goles marcados por partido en italia  Country
it ITA    1.258
Name: GPP, dtype: float64
Media de goles marcados por partidos en españa Country
es ESP    1.210417
Name: GPP, dtype: float64


C:\Users\marcg\AppData\Local\Temp\ipykernel_1176\1547584608.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ita_df["GPP"]=ita_df["GF"]/ita_df["MP"]
C:\Users\marcg\AppData\Local\Temp\ipykernel_1176\1547584608.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  esp_df["GPP"]=esp_df["GF"]/esp_df["MP"]
C:\Users\marcg\AppData\Local\Temp\ipykernel_1176\1547584608.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

Veo que en italia se meten menos de dos goles por partido, voy a comprobar si esa diferencia es significativa, para ello voy a realizar un test de hipotesis:
Ho (hipotesis nula): La media de goles marcados por patidos es igual a dos
Ha (hipotesis alternativa):La media de goles marcados por patidos es menor a dos

In [5]:
from pingouin import ttest

results=ttest(ita_df["GPP"],2,alternative='less')

print(results)

               T  dof alternative         p-val         CI95%   cohen-d  \
T-test -7.635297   19        less  1.663923e-07  [-inf, 1.43]  1.707304   

             BF10  power  
T-test  9.831e+04    1.0  


Hemos obtenido un p-val inferior a 0.05 por lo tanto podemos rechazar la hipotesis nula, podemos ver que hay una diferencia significativa de goles marcados por patidos.

Veo que en españa se meten menos goles que en Italia, voy a comprobar si esa diferencia es significativa. Para ello vamos a realizar un test de hipotesis:
Ho (hipotesis nula): La media de goles por partido de españa es la misma que en Italia
Ha (hipotesis alternativa): La media de goles por partido en españa es menor que en Italia

In [6]:
from pingouin import ttest

results=ttest(esp_df["GPP"],ita_df["GPP"],alternative='less')

print(results)

               T  dof alternative    p-val         CI95%   cohen-d   BF10  \
T-test -0.381333   38        less  0.35254  [-inf, 0.16]  0.120588  0.655   

           power  
T-test  0.101995  


Hemos obtenido un p-val superior al 0.05 por lo tanto no podemos rechazar la hipotesis nula, a nivel matematico no hay diferencia significativas entre los goles por partido que se realizan en españa que en italia 

In [12]:
print(futbol_df.groupby('Country')['GPP'].mean())

Country
de GER     1.579710
eng ENG    1.356728
es ESP     1.210417
fr FRA     1.419231
it ITA     1.258000
Name: GPP, dtype: float64


Veo que en una liga se meten mas goles que en otras ligas, voy a comprobar si esa diferencia es significativa. Para ello vamos a realizar un test de hipotesis:
Ho (hipotesis nula): La media de goles por partido es la misma en todas las ligas
Ha (hipotesis alternativa): La media de goles por partido en una liga es mayor que en las otras ligas

In [15]:
from pingouin import anova
anova_results = anova(data=futbol_df, dv='GPP', between='Country')

print(anova_results)

    Source  ddof1  ddof2         F     p-unc       np2
0  Country      4     93  1.958386  0.107339  0.077688


veo que el p-value es mayor de 0.05, por lo tanto no se rechaza la hipotesis nula, se marca lo mismo en todas las ligas

In [16]:
from pingouin import pairwise_tests

pair_results = pairwise_tests(data=futbol_df, dv='GPP', between='Country')
print(pair_results)

  Contrast        A        B  Paired  Parametric         T        dof  \
0  Country   de GER  eng ENG   False        True  1.372091  35.992496   
1  Country   de GER   es ESP   False        True  2.721908  31.236329   
2  Country   de GER   fr FRA   False        True  1.069420  35.243214   
3  Country   de GER   it ITA   False        True  2.182646  34.790722   
4  Country  eng ENG   es ESP   False        True  1.028032  38.000000   
5  Country  eng ENG   fr FRA   False        True -0.400405  38.000000   
6  Country  eng ENG   it ITA   False        True  0.643032  38.000000   
7  Country   es ESP   fr FRA   False        True -1.632363  38.000000   
8  Country   es ESP   it ITA   False        True -0.381333  38.000000   
9  Country   fr FRA   it ITA   False        True  1.149262  38.000000   

  alternative     p-unc   BF10    hedges  
0   two-sided  0.178527  0.657  0.433557  
1   two-sided  0.010520  5.013  0.879319  
2   two-sided  0.292150  0.494  0.340864  
3   two-sided  0.035901 

In [17]:
print(futbol_df.groupby('Last 5')['GPP'].mean())

Last 5
D D D L L    1.458333
D D D W W    0.782609
D D L L D    0.440000
D D L L L    0.807692
D D L L W    0.960000
               ...   
W W L W W    2.869565
W W W L D    2.115385
W W W L W    1.916667
W W W W L    1.960000
W W W W W    2.043478
Name: GPP, Length: 80, dtype: float64


In [31]:
futbol_anterior_df=pd.read_csv("Futbol_season_anterior.csv")


futbol_anterior_df["GPP"]=futbol_anterior_df["GF"]/futbol_anterior_df["MP"]

print(futbol_anterior_df['GPP'].mean())
print(futbol_df['GPP'].mean())

1.4082264484741267
1.3604315973881191


Veo que en respecto al año pasado, se han metido menos goles, voy a comprobar si esa diferencia es significativa. Para ello vamos a realizar un test de hipotesis:
Ho (hipotesis nula): La media de goles por partido es la misma que el año pasado
Ha (hipotesis alternativa): La media de goles por partido respecto al año pasado es menor

In [33]:
sults=ttest(futbol_df["GPP"],futbol_anterior_df["GPP"],alternative='less')

print(results)

               T  dof alternative    p-val         CI95%   cohen-d   BF10  \
T-test -0.381333   38        less  0.35254  [-inf, 0.16]  0.120588  0.655   

           power  
T-test  0.101995  


veo que el p-value es mayor de 0.05, por lo tanto no se rechaza la hipotesis nula, respecto al año pasado se han marcado lo misma cantidad